##  Cofactor Swap

In [1]:
#In this script we analyse if swapping the cofactors has a postive effect on the resveratrol production
from cobra.io import read_sbml_model
from cameo.core import manipulation
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization

In [2]:
# Verify working directory and change it if needed
import os
os.getcwd()
os.chdir("C:/Users/Deborah/anaconda3/envs/cellfactorydesign/27410-group-assigment-group-4-resveratrol-in-s-cerevisiae/") #change accordingly

In [3]:
model = read_sbml_model("data/models/yeast8_resv_glc.xml")

In [4]:
manipulation.increase_flux(model.reactions.EX_glc__D_e, -1.0, -4.0)

biomass_objective=model.optimize().objective_value
model.reactions.GROWTH.lower_bound = biomass_objective*0.5

print('Glucose bounds: ', model.reactions.EX_glc__D_e.bounds)
print('Biomass bounds: ', model.reactions.GROWTH.bounds)

Glucose bounds:  (-4.0, -4.0)
Biomass bounds:  (0.16850811270765662, 1000.0)


In [5]:
model.objective=model.reactions.VVVST1
model.optimize()

,fluxes,reduced_costs
D_LACDcm,0.000000,-5.551115e-17
D_LACDm,0.000000,6.938894e-18
BTDD_RR,0.000000,0.000000e+00
L_LACD2cm,0.000000,6.938894e-18
13BETGLUSYN,0.126131,5.551115e-17
...,...,...
r_4710,0.000000,0.000000e+00
HA-TAL,0.638618,0.000000e+00
AT-4CL1,0.638618,3.330669e-15
VVVST1,0.638618,2.775558e-16


In [6]:
# we choose the yield of resveratrol on glucose to be the objective for the simulations
py = product_yield(model.reactions.VVVST1, model.reactions.EX_glc__D_e)
optswap = CofactorSwapOptimization(model=model, objective_function=py)

In [7]:
%time
result=optswap.run(max_evaluations=2000, max_size=2)
result

Wall time: 0 ns
Starting optimization at Tue, 29 Nov 2022 15:14:39


HBox()

Finished after 00:05:01


,index,targets,fitness
0,0,"(ALCD2x,)",1.007638
1,1,"(ICDHyr,)",1.007638
2,2,"(GAPD,)",0.644403
3,3,"(C3STEDEH4MET, SHK3Dr)",0.615868
4,4,"(SHK3Dr,)",0.615835
5,5,"(FATACYCOASYNNC16,)",0.614079
6,6,"(FATACYCOASYNNC18,)",0.613044
7,7,"(PGCD,)",0.612949
8,8,"(HMGCOAR,)",0.612410
9,9,"(IPMD,)",0.612317


#### Flux Balance Analysis for swapping the cofactors in each reaction

In [8]:
# Show the top 3 reactions:
model.reactions.get_by_id('ALCD2x')

Reaction identifier,ALCD2x
Name,alcohol dehydrogenase (ethanol to acetaldehyde)
Memory address,0x01990626cf48
Stoichiometry,etoh_c + nad_c --> acald_c + h_c + nadh_c ethanol + NAD --> acetaldehyde + H+ + NADH
GPR,YMR303C
Lower bound,0.0
Upper bound,1000.0


In [9]:
model.reactions.get_by_id('ICDHyr')

Reaction identifier,ICDHyr
Name,isocitrate dehydrogenase (NADP)
Memory address,0x0199065e5e88
Stoichiometry,icit_c + nadp_c <=> akg_c + co2_c + nadph_c isocitrate + NADP(+) <=> 2-oxoglutarate + carbon dioxide + NADPH
GPR,YLR174W
Lower bound,-1000.0
Upper bound,1000.0


In [10]:
model.reactions.get_by_id('GAPD')

Reaction identifier,GAPD
Name,glyceraldehyde-3-phosphate dehydrogenase
Memory address,0x0199064a8fc8
Stoichiometry,"g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c glyceraldehyde 3-phosphate + NAD + phosphate <=> 1,3-bisphospho-D-glycerate + H+ + NADH"
GPR,YGR192C or YJL052W or YJR009C
Lower bound,-1000.0
Upper bound,1000.0


In [ ]:
with model:
    fba_r=fba(model, objective=model.reactions.GROWTH)
    fba_ref=fba_r.data_frame
    fba_ref.sort_values(by='flux', ascending=False, inplace=True)
    
    print('flux of ALCD2x: ', fba_ref.loc['ALCD2x'][0], 'mmol/gDW*h')
    print('flux of ICDHyr: ', fba_ref.loc['ICDHyr'][0], 'mmol/gDW*h')
    print('flux of GAPD: ', fba_ref.loc['GAPD'][0], 'mmol/gDW*h')
    
    manipulation.swap_cofactors(model.reactions.get_by_id('ALCD2x'), model, swap_pairs=('nad_c', 'nadh_c', 'nadp_c', 'nadph_c'))
    manipulation.swap_cofactors(model.reactions.get_by_id('ICDHyr'), model, swap_pairs=('nad_c', 'nadh_c', 'nadp_c', 'nadph_c'))
    manipulation.swap_cofactors(model.reactions.get_by_id('GAPD'), model, swap_pairs=('nad_c', 'nadh_c', 'nadp_c', 'nadph_c'))
    
    fba_cs=fba(model, objective=model.reactions.GROWTH)
    fba_swap=fba_cs.data_frame
    fba_swap.sort_values(by='flux', ascending=False, inplace=True)
    print('flux of ALCD2x: ', fba_swap.loc['ALCD2x'][0], 'mmol/gDW*h')
    print('flux of ICDHyr: ', fba_swap.loc['ICDHyr'][0], 'mmol/gDW*h')
    print('flux of GAPD: ', fba_swap.loc['GAPD'][0], 'mmol/gDW*h')